0. flow 전처리 (IQR, Savgol)
1. weather 전처리 
2. flow & weather merge
3. sliding window 
4. Train / Val / Test Split (0.7 / 0.15 / 0.15)
5. 정규화 (Train기준)
6. 모델 생성 LSTMSeq2SeqAttnModel 
7~8. 학습 (patience=10, N=3) → 평가

In [1]:
# ============================================================
# 라이브러리 임포트 & 재현성 시드 설정
# ============================================================
# - N_RUNS: 동일 모델을 서로 다른 시드로 N회 반복 학습 (안정성 검증)
# - set_seed(): 모든 난수 생성기(Python, NumPy, PyTorch, CUDA)를
#   동일 시드로 고정하여 실험 재현성 보장
# ============================================================

from pathlib import Path
import numpy as np
import pandas as pd
import random, os, copy

N_RUNS = 3
SEEDS = [42, 123, 7]

def set_seed(seed):
    random.seed(seed)                          # Python 기본 난수
    np.random.seed(seed)                       # NumPy 난수
    import torch
    torch.manual_seed(seed)                    # PyTorch CPU 난수
    torch.cuda.manual_seed_all(seed)           # PyTorch GPU 난수 (멀티 GPU 포함)
    torch.backends.cudnn.deterministic = True  # cuDNN 결정론적 연산 강제
    torch.backends.cudnn.benchmark = False     # 자동 최적화 비활성화 (재현성 우선)
    os.environ['PYTHONHASHSEED'] = str(seed)   # Python hash 시드 고정

print(f"Multi-run: {N_RUNS} runs, seeds={SEEDS}")

Multi-run: 3 runs, seeds=[42, 123, 7]


### Data Split 구조 (4계절 Block Sampling, 60일 x 4)

각 블록(60일)을 시간 순서대로 70 / 15 / 15로 분할 (블록 간 87분 gap으로 data leak 방지)

| 계절 | Train (~42일) | Val (~9일) | Test (~9일) |
|------|--------------|-------------|--------------|
| Winter | 23/12/01 ~ 24/01/11 | 24/01/11 ~ 24/01/20 | 24/01/20 ~ 24/01/29 |
| Spring | 24/03/15 ~ 24/04/25 | 24/04/25 ~ 24/05/04 | 24/05/04 ~ 24/05/13 |
| Summer | 23/07/01 ~ 23/08/11 | 23/08/11 ~ 23/08/20 | 23/08/20 ~ 23/08/29 |
| Fall | 23/09/15 ~ 23/10/26 | 23/10/26 ~ 23/11/04 | 23/11/04 ~ 23/11/13 |

**예측 구조:** 72분 입력 (10 features) -> 15분 예측 (flow value) x 4 rolling = 1시간

**Loss:** Step-weighted MSE (step 1=1.0 -> step 15=2.0)
**LR Scheduler:** ReduceLROnPlateau (factor=0.5, patience=3)

## 0. flow, weather 데이터 Load

In [2]:
# ============================================================
# [데이터 로드] Flow 유량 데이터 + Weather 파일 경로 설정
# ============================================================

BASE_DIR = Path.cwd().parent

# Flow raw data 로드 (J배수지, reservoir/10.csv)
flow_file = BASE_DIR / "data" / "rawdata" / "reservoir" / "10.csv"
df_flow = pd.read_csv(
    flow_file,
    header=None,
    usecols=[1, 2],
    names=['time', 'value']
).sort_values('time').reset_index(drop=True)

df_flow['time'] = pd.to_datetime(df_flow['time'], format='mixed', errors='coerce')
df_flow = df_flow.dropna(subset=['time'])
print(f"Flow 원본: {len(df_flow):,}개 ({df_flow['time'].min()} ~ {df_flow['time'].max()})")

# Weather 파일 경로
weather_file = BASE_DIR / "data" / "rawdata" / "weather"

Flow 원본: 943,434개 (2023-01-01 00:01:00 ~ 2024-10-17 17:19:00)


## 0. Flow 전처리
- IQR 이상치 + 음수 + 급변동 제거 → NaN
- Linear interpolation (양방향)
- Savitzky-Golay filter (window=51, polyorder=2)

In [3]:
# ============================================================
# [Flow 전처리] 이상치 제거 → 보간 → 스무딩
# ============================================================
# 1단계: IQR 이상치 + 음수값 + 급변동(상위 0.1%) → NaN 마스킹
# 2단계: 양방향 선형 보간(interpolate) + 음수 클리핑
# 3단계: Savitzky-Golay 필터로 고주파 노이즈 제거
#        (window=51분, 2차 다항식 → 추세 보존 + 노이즈 억제)
# ============================================================

from scipy.signal import savgol_filter

# --- 1단계: 이상치 탐지 & NaN 마스킹 ---
Q1 = df_flow['value'].quantile(0.25)
Q3 = df_flow['value'].quantile(0.75)
IQR = Q3 - Q1

iqr_mask = (df_flow['value'] < Q1 - 1.5 * IQR) | (df_flow['value'] > Q3 + 1.5 * IQR)  # IQR 범위 밖
negative_mask = df_flow['value'] < 0                          # 음수 (물리적 불가)
diff = df_flow['value'].diff().abs()
spike_threshold = diff.quantile(0.999)
spike_mask = diff > spike_threshold                            # 급변동 (상위 0.1%)

total_mask = iqr_mask | negative_mask | spike_mask             # 세 조건 OR → NaN 처리
df_flow.loc[total_mask, 'value'] = np.nan

print(f"이상치 제거: IQR={iqr_mask.sum()}, 음수={negative_mask.sum()}, "
      f"급변동(>{spike_threshold:.2f})={spike_mask.sum()}, 총={total_mask.sum()}")

# --- 2단계: 선형 보간 + 음수 클리핑 ---
df_flow['value'] = df_flow['value'].interpolate(method='linear', limit_direction='both')
df_flow['value'] = df_flow['value'].clip(lower=0)  # 보간 결과 음수 방지

# --- 3단계: Savitzky-Golay 스무딩 필터 ---
df_flow['value'] = savgol_filter(df_flow['value'], window_length=51, polyorder=2)
df_flow['value'] = df_flow['value'].clip(lower=0)  # 필터 결과 음수 방지

print(f"전처리 완료: {len(df_flow):,}개, "
      f"범위 [{df_flow['value'].min():.2f}, {df_flow['value'].max():.2f}]")

이상치 제거: IQR=21, 음수=0, 급변동(>91.06)=944, 총=962
전처리 완료: 943,434개, 범위 [0.00, 325.52]


In [4]:
# ============================================================
# [저장] 전처리 완료된 Flow 데이터를 processed/ 폴더에 CSV로 저장
# ============================================================
# - 원본(data/rawdata/)은 수정하지 않고, 전처리 결과만 별도 저장
# - 이후 재실행 시 이 파일을 로드하면 전처리 단계 스킵 가능
# ============================================================

save_path = BASE_DIR / "data" / "processed" / "flow_preprocessed.csv"
df_flow.to_csv(save_path, index=False)
print(f"저장 완료: {save_path} ({len(df_flow):,}행)")

저장 완료: c:\Users\user\Documents\Main\wsl_folder\work\pro\data\processed\flow_preprocessed.csv (943,434행)


- weather 데이터 인코딩, concat

In [5]:
# ============================================================
# [Weather 로드] 기상 CSV 파일 다중 인코딩 시도 후 concat
# ============================================================
# - 기상청 CSV는 인코딩이 파일마다 다를 수 있음 (euc-kr/cp949/utf-8)
# - read_weather_csv(): 4가지 인코딩을 순차 시도하여 자동 감지
# - weather 폴더 내 SYNOP CSV만 시간순 정렬 → 하나의 DataFrame으로 병합
#   (weather_total_raw.csv 등 파생 파일은 제외)
# ============================================================

def read_weather_csv(f):
    """다중 인코딩 시도로 기상 CSV 1개 파일을 읽는다."""
    for enc in ["euc-kr", "cp949", "utf-8", "utf-8-sig"]:
        try:
            return pd.read_csv(f, encoding=enc)
        except (UnicodeDecodeError, UnicodeError):
            continue
    raise ValueError(f"인코딩 실패: {f.name}")

# SYNOP 원본 파일만 선택 (weather_total_raw.csv 등 파생 파일 제외)
files = sorted([f for f in weather_file.glob("*.csv") if f.name.startswith("SYNOP")])
print(f"읽을 파일 수: {len(files)}")

weather = pd.concat(                              # 전체 파일을 하나로 병합
    [read_weather_csv(f) for f in files],
    ignore_index=True
)
print(f"concat 후 행 수: {len(weather):,}")

읽을 파일 수: 22
concat 후 행 수: 799,438


- 데이터 저장

In [6]:
# [저장] Weather 원본 concat 결과를 processed/에 보관 (원본 보호)
weather.to_csv(BASE_DIR / "data" / "processed" / "weather_total_raw.csv", index=False, encoding="utf-8-sig")

## 1. weather 전처리

In [7]:
# ============================================================
# [Weather 전처리] 결측 보간, Rainfall 차분 변환, 세그먼트 ID 부여
# ============================================================

# --- 1) datetime 변환 (format='mixed'로 다양한 형식 허용, pandas 3.0 호환) ---
weather['일시'] = pd.to_datetime(weather['일시'], format='mixed', errors='coerce')
before_dt = len(weather)
weather = weather.dropna(subset=['일시'])
dropped = before_dt - len(weather)
print(f"datetime 파싱: {len(weather):,}행 성공" + (f", {dropped}행 제거" if dropped else ""))

# --- 2) 결측 처리 ---
weather['0.5mm 일 누적 강수량(mm)'] = weather['0.5mm 일 누적 강수량(mm)'].fillna(0)
weather['기온(℃)'] = weather['기온(℃)'].interpolate(method='linear', limit=60)
weather['상대습도(%)'] = weather['상대습도(%)'].interpolate(method='linear', limit=60)

# --- 3) Rainfall 차분 변환: 일 누적 → 분당 변화량 ---
rainfall_raw = weather['0.5mm 일 누적 강수량(mm)'].copy()
rainfall_diff = rainfall_raw.diff().fillna(0).clip(lower=0)

print(f"Rainfall 변환: 누적 → 차분")
print(f"  원본 범위: [{rainfall_raw.min():.2f}, {rainfall_raw.max():.2f}]")
print(f"  차분 범위: [{rainfall_diff.min():.4f}, {rainfall_diff.max():.4f}]")
print(f"  비영점 비율: {(rainfall_diff > 0).mean()*100:.1f}%")

weather['0.5mm 일 누적 강수량(mm)'] = rainfall_diff

# --- 4) 장기 결측(>60분 gap) 행 제거 ---
before = len(weather)
weather = weather.dropna(subset=['기온(℃)', '상대습도(%)'])
print(f'장기 결측 제거: {before:,} -> {len(weather):,} ({len(weather)/before*100:.1f}% 유지)')

# --- 5) 컬럼명 영문 변환 ---
weather = weather.rename(columns={
    '일시': 'datetime',
    '기온(℃)': 'temperature',
    '0.5mm 일 누적 강수량(mm)': 'rainfall',
    '상대습도(%)': 'humidity',
})

# 중복 timestamp 제거
weather = weather.drop_duplicates(subset='datetime', keep='first')
weather = weather.sort_values('datetime').reset_index(drop=True)

# --- 6) segment_id 부여 ---
time_diff = weather['datetime'].diff()
seg_boundary = time_diff > pd.Timedelta(minutes=1)
weather['segment_id'] = seg_boundary.cumsum()
print(f'연속 세그먼트 수: {weather["segment_id"].nunique()}')

print(weather.shape)
print(weather.head())

datetime 파싱: 799,435행 성공, 3행 제거
Rainfall 변환: 누적 → 차분
  원본 범위: [0.00, 160.00]
  차분 범위: [0.0000, 119.0000]
  비영점 비율: 0.4%
장기 결측 제거: 799,435 -> 762,326 (95.4% 유지)
연속 세그먼트 수: 3284
(762326, 5)
             datetime  temperature  rainfall  humidity  segment_id
0 2023-01-01 00:01:00         -3.3       0.0      91.6           0
1 2023-01-01 00:02:00         -3.3       0.0      91.6           0
2 2023-01-01 00:03:00         -3.3       0.0      91.6           0
3 2023-01-01 00:04:00         -3.2       0.0      91.6           0
4 2023-01-01 00:05:00         -3.2       0.0      91.6           0


In [8]:
# [확인] Weather 기술통계 — 각 feature의 분포/범위 점검
weather.describe()

,datetime,temperature,rainfall,humidity,segment_id
count,762326,762326.000000,762326.000000,762326.000000,762326.000000
mean,2023-12-03 01:55:51.533543,15.311656,0.002976,72.047670,1438.558028
min,2023-01-01 00:01:00,-16.500000,0.000000,2.900000,0.000000
25%,2023-06-23 20:18:15,6.200000,0.000000,57.400000,436.000000
50%,2023-12-07 15:16:30,16.400000,0.000000,76.300000,931.000000
75%,2024-05-15 10:50:45,24.600000,0.000000,90.500000,2862.000000
max,2024-10-25 15:10:00,37.700000,119.000000,99.900000,3283.000000
std,NaN,10.906573,0.165693,21.186558,1202.060302


In [9]:
# [확인] 결측값(NaN) 잔존 여부 체크 — 전처리 후 0이어야 정상
df = pd.DataFrame(weather)
df.isnull().sum()

datetime       0
temperature    0
rainfall       0
humidity       0
segment_id     0
dtype: int64

In [10]:
# [컬럼 선택] 학습에 사용할 Weather 컬럼만 추출 (+ segment_id)
df_weather = df[['datetime', 'temperature', 'rainfall', 'humidity', 'segment_id']].copy()

## 2. weather data merge & weather, time feature 추가

In [11]:
# [Merge] Flow + Weather를 timestamp(time ↔ datetime) 기준 inner join
# - inner join이므로 양쪽 모두 존재하는 시각만 남음
df_merged = pd.merge(df_flow, df_weather, how='inner', left_on='time', right_on="datetime")

In [12]:
# ============================================================
# [후처리] merge 후 정리 + Cyclical Temporal Features 생성
# ============================================================
# 1) 중복 datetime 컬럼 제거, 정렬, NaN 행 제거
# 2) segment_id 재계산: merge로 행이 빠졌으므로 불연속 경계 재탐지
# 3) Cyclical encoding: sin/cos 변환으로 주기성을 [0,1] 범위 연속값으로 표현
#    - time_sin/cos : 하루 주기 (T=1440분)  → 시간대별 유량 패턴
#    - dow_sin/cos  : 주간 주기 (T=7일)     → 평일/주말 패턴
#    - season_sin/cos: 연간 주기 (T=365.25일) → 계절별 패턴
#    0.5*sin()+0.5 변환으로 [0,1] 범위 유지 (MinMaxScaler 불필요)
# ============================================================

df_merged = df_merged.drop(columns=['datetime'])  # merge로 생긴 중복 컬럼 제거

df_merged = df_merged.sort_values('time').reset_index(drop=True)
df_merged = df_merged.dropna()
print(f"데이터: {len(df_merged):,}개 (1분 단위)")

# --- segment_id 재계산 (merge 후 실제 불연속 기준) ---
time_diff_merged = df_merged['time'].diff()
seg_boundary_merged = time_diff_merged > pd.Timedelta(minutes=1)
df_merged['segment_id'] = seg_boundary_merged.cumsum()
print(f"merge 후 연속 세그먼트 수: {df_merged['segment_id'].nunique()}")

# --- Cyclical Temporal Features ---
t: pd.Series = df_merged['time']

hour = t.dt.hour.astype(np.float64)
minute = t.dt.minute.astype(np.float64)
dow = t.dt.dayofweek.astype(np.float64)
doy = t.dt.dayofyear.astype(np.float64)

# 하루 주기 (분 단위, T=1440분)
minute_of_day = hour * 60 + minute
df_merged['time_sin'] = 0.5 * np.sin(2 * np.pi * minute_of_day / 1440) + 0.5
df_merged['time_cos'] = 0.5 * np.cos(2 * np.pi * minute_of_day / 1440) + 0.5

# 주간 주기 (요일, T=7)
df_merged['dow_sin'] = 0.5 * np.sin(2 * np.pi * dow / 7) + 0.5
df_merged['dow_cos'] = 0.5 * np.cos(2 * np.pi * dow / 7) + 0.5

# 연간 주기 (일수, T=365.25)
df_merged['season_sin'] = 0.5 * np.sin(2 * np.pi * doy / 365.25) + 0.5
df_merged['season_cos'] = 0.5 * np.cos(2 * np.pi * doy / 365.25) + 0.5

print(f"Temporal features 추가 완료: {df_merged.shape}")
print(df_merged.head())

데이터: 750,296개 (1분 단위)
merge 후 연속 세그먼트 수: 3789
Temporal features 추가 완료: (750296, 12)
                 time      value  temperature  rainfall  humidity  segment_id  \
0 2023-01-01 00:01:00  96.577302         -3.3       0.0      91.6           0   
1 2023-01-01 00:02:00  96.739744         -3.3       0.0      91.6           0   
2 2023-01-01 00:03:00  96.895855         -3.3       0.0      91.6           0   
3 2023-01-01 00:04:00  97.045636         -3.2       0.0      91.6           0   
4 2023-01-01 00:05:00  97.189086         -3.2       0.0      91.6           0   

   time_sin  time_cos   dow_sin   dow_cos  season_sin  season_cos  
0  0.502182  0.999995  0.109084  0.811745    0.508601    0.999926  
1  0.504363  0.999981  0.109084  0.811745    0.508601    0.999926  
2  0.506545  0.999957  0.109084  0.811745    0.508601    0.999926  
3  0.508726  0.999924  0.109084  0.811745    0.508601    0.999926  
4  0.510907  0.999881  0.109084  0.811745    0.508601    0.999926  


In [13]:
# [확인] merge 결과 shape & dtype 점검
df_merged.shape, df_merged.info()

<class 'pandas.DataFrame'>
RangeIndex: 750296 entries, 0 to 750295
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   time         750296 non-null  datetime64[us]
 1   value        750296 non-null  float64       
 2   temperature  750296 non-null  float64       
 3   rainfall     750296 non-null  float64       
 4   humidity     750296 non-null  float64       
 5   segment_id   750296 non-null  int64         
 6   time_sin     750296 non-null  float64       
 7   time_cos     750296 non-null  float64       
 8   dow_sin      750296 non-null  float64       
 9   dow_cos      750296 non-null  float64       
 10  season_sin   750296 non-null  float64       
 11  season_cos   750296 non-null  float64       
dtypes: datetime64[us](1), float64(10), int64(1)
memory usage: 68.7 MB


((750296, 12), None)

In [14]:
# [확인] merge 후 전체 기술통계 — 각 feature 분포 점검
df_merged.describe()

,time,value,temperature,rainfall,humidity,segment_id,time_sin,time_cos,dow_sin,dow_cos,season_sin,season_cos
count,750296,750296.000000,750296.000000,750296.000000,750296.000000,750296.000000,750296.000000,750296.000000,750296.000000,750296.000000,7.502960e+05,750296.000000
mean,2023-11-28 04:26:42.255643,151.178574,15.306791,0.002967,71.904041,1667.789278,0.503187,0.498909,0.497432,0.497254,5.241825e-01,0.470597
min,2023-01-01 00:01:00,0.000000,-16.500000,0.000000,2.900000,0.000000,0.000000,0.000000,0.012536,0.049516,2.889877e-07,0.000010
25%,2023-06-21 20:52:45,108.331885,6.100000,0.000000,57.300000,562.000000,0.149545,0.144907,0.109084,0.049516,1.605898e-01,0.134311
50%,2023-12-03 15:42:30,143.890453,16.400000,0.000000,76.100000,1171.000000,0.508726,0.497818,0.500000,0.388740,5.771020e-01,0.440471
75%,2024-05-09 11:17:15,192.041955,24.700000,0.000000,90.300000,3139.000000,0.856625,0.852007,0.890916,0.811745,8.768488e-01,0.817949
max,2024-10-17 17:19:00,325.516638,37.700000,119.000000,99.900000,3788.000000,1.000000,0.999995,0.987464,1.000000,9.999928e-01,0.999926
std,NaN,56.916589,10.971803,0.166925,21.225753,1331.184225,0.353635,0.353456,0.354114,0.352972,3.560774e-01,0.348941


In [15]:
# ============================================================
# [Block Sampling] 4계절 × 60일 블록 추출
# ============================================================
# - 전체 데이터에서 계절별 대표 구간(60일)을 선택하여 균형 잡힌 학습 데이터 구성
# - 2023~2024년 교차 배치로 연도 편향 방지
# - block_id: 각 계절 블록의 식별자 (0=겨울, 1=봄, 2=여름, 3=가을)
# ============================================================

season_ranges = [
    ('2023-12-01', '2024-01-29', 'Winter'),   # block_id=0
    ('2024-03-15', '2024-05-13', 'Spring'),    # block_id=1
    ('2023-07-01', '2023-08-29', 'Summer'),    # block_id=2
    ('2023-09-15', '2023-11-13', 'Fall'),      # block_id=3
]

blocks = []
for i, (start, end, name) in enumerate(season_ranges):
    mask = (df_merged['time'] >= start) & (df_merged['time'] <= end)
    block = df_merged[mask].copy()
    block['block_id'] = i                      # 계절 블록 식별자
    blocks.append(block)
    print(f"[{name}] {start} ~ {end}: {len(block):,} rows")

df_merged = pd.concat(blocks, ignore_index=True)
print(f"\nTotal: {len(df_merged):,} rows (4-season block sampling, 60일 × 4)")

[Winter] 2023-12-01 ~ 2024-01-29: 71,779 rows
[Spring] 2024-03-15 ~ 2024-05-13: 69,639 rows
[Summer] 2023-07-01 ~ 2023-08-29: 70,804 rows
[Fall] 2023-09-15 ~ 2023-11-13: 68,351 rows

Total: 280,573 rows (4-season block sampling, 60일 × 4)


## 3. Sliding Windows 생성
- X: (n_samples, input_time, 10) — value, temperature, rainfall, humidity + 6 cyclical features
- y: (n_samples, output_time) — value만 예측

In [16]:
# ============================================================
# [설정] Sliding Window 입출력 정의 & Feature 컬럼 지정
# ============================================================
# feature_cols (10개): 모델 입력 X의 컬럼 순서
#   - 물리량 4개: value, temperature, rainfall, humidity
#   - 시간 인코딩 6개: time_sin/cos, dow_sin/cos, season_sin/cos
# scale_cols (4개): MinMaxScaler 적용 대상 (sin/cos는 이미 [0,1])
# target_col: 예측 대상 = value (유량)
# input_time=72: 과거 72분(1.2시간) 입력
# output_time=15: 미래 15분 예측 (× 4 rolling = 1시간)
# ============================================================

feature_cols = ['value', 'temperature', 'rainfall', 'humidity',
                'time_sin', 'time_cos', 'dow_sin', 'dow_cos',
                'season_sin', 'season_cos']
scale_cols = ['value', 'temperature', 'rainfall', 'humidity']
target_col = 'value'
input_time = 72    # 72 steps × 1min = 72min
output_time = 15   # 15 steps × 1min = 15min (× 4 rolling = 1h)

print(f"=== Settings ===")
print(f"Input: {input_time} steps ({input_time}min = {input_time/60:.1f}h), {len(feature_cols)} features")
print(f"Output: {output_time} steps ({output_time}min) × 4 rolling = {output_time*4}min")

=== Settings ===
Input: 72 steps (72min = 1.2h), 10 features
Output: 15 steps (15min) × 4 rolling = 60min


## 4. Train / Val / Test Split (0.7 / 0.15 / 0.15)
- Split 경계에 gap 적용 → sliding window 겹침(data leak) 방지

In [17]:
# Sliding window 생성 + Train/Val/Test split (block × segment 독립)
# ★ 수정: segment_id 경계를 넘는 window 생성 방지
# split 경계에 gap 적용 → sliding window 겹침(data leak) 방지

train_ratio, val_ratio = 0.7, 0.15
split_gap = input_time + output_time  # 87 steps gap (data leak 방지)
min_segment_len = input_time + output_time  # 세그먼트 최소 길이

X_train_list, y_train_list = [], []
X_val_list, y_val_list = [], []
X_test_list, y_test_list = [], []
test_times_list = []

skipped_segments = 0
used_segments = 0

for block_id in sorted(df_merged['block_id'].unique()):
    block = df_merged[df_merged['block_id'] == block_id]
    
    # ★ block 내 segment별로 sliding window 생성
    seg_X, seg_y, seg_times = [], [], []
    
    for seg_id in sorted(block['segment_id'].unique()):
        segment = block[block['segment_id'] == seg_id].reset_index(drop=True)
        
        if len(segment) < min_segment_len:
            skipped_segments += 1
            continue  # 너무 짧은 세그먼트 스킵
        
        used_segments += 1
        seg_features = segment[feature_cols].values.astype(np.float32)
        seg_target = segment[target_col].values.astype(np.float32)
        seg_time = segment['time'].values
        
        n_samples = len(segment) - input_time - output_time + 1
        for i in range(n_samples):
            seg_X.append(seg_features[i : i + input_time])
            seg_y.append(seg_target[i + input_time : i + input_time + output_time])
            seg_times.append(seg_time[i + input_time])
    
    if len(seg_X) == 0:
        print(f"Block {block_id}: 유효 윈도우 없음 (스킵)")
        continue
    
    X_block = np.array(seg_X)
    y_block = np.array(seg_y)
    times_block = np.array(seg_times)
    n_samples = len(X_block)

    # gap을 두어 Train/Val/Test 간 window 겹침 방지
    t_end = int(n_samples * train_ratio)
    v_start = t_end + split_gap
    v_end = v_start + int(n_samples * val_ratio)
    test_start = v_end + split_gap

    X_train_list.append(X_block[:t_end])
    y_train_list.append(y_block[:t_end])
    X_val_list.append(X_block[v_start:v_end])
    y_val_list.append(y_block[v_start:v_end])
    X_test_list.append(X_block[test_start:])
    y_test_list.append(y_block[test_start:])

    for i in range(test_start, n_samples):
        test_times_list.append(times_block[i])

    n_train = t_end
    n_val = v_end - v_start
    n_test = n_samples - test_start
    print(f"Block {block_id}: {n_samples:,} samples "
          f"(train={n_train:,} / val={n_val:,} / test={n_test:,}) "
          f"gap={split_gap}")

X_train = np.concatenate(X_train_list)
y_train = np.concatenate(y_train_list)
X_val = np.concatenate(X_val_list)
y_val = np.concatenate(y_val_list)
X_test = np.concatenate(X_test_list)
y_test = np.concatenate(y_test_list)
test_times = np.array(test_times_list)

n_total = len(X_train) + len(X_val) + len(X_test)
print(f"\n=== Sliding Window + Split (Seasonal Blocks × Segments, gap={split_gap}) ===")
print(f"★ Segment-aware: {used_segments}개 세그먼트 사용, {skipped_segments}개 스킵 (< {min_segment_len} steps)")
print(f"X shape: ({input_time}, {len(feature_cols)}), y shape: ({output_time},)")
print(f"Train: {len(X_train):,} | Val: {len(X_val):,} | Test: {len(X_test):,} | Total: {n_total:,}")
print(f"Memory: X_train {X_train.nbytes/1e6:.1f} MB, y_train {y_train.nbytes/1e6:.1f} MB")

Block 0: 37,534 samples (train=26,273 / val=5,630 / test=5,457) gap=87
Block 1: 47,518 samples (train=33,262 / val=7,127 / test=6,955) gap=87
Block 2: 64,694 samples (train=45,285 / val=9,704 / test=9,531) gap=87
Block 3: 56,652 samples (train=39,656 / val=8,497 / test=8,325) gap=87

=== Sliding Window + Split (Seasonal Blocks × Segments, gap=87) ===
★ Segment-aware: 533개 세그먼트 사용, 1487개 스킵 (< 87 steps)
X shape: (72, 10), y shape: (15,)
Train: 144,476 | Val: 30,958 | Test: 30,268 | Total: 205,702
Memory: X_train 416.1 MB, y_train 8.7 MB


## 5. 정규화 (Train 기준)
- X: feature별 개별 MinMaxScaler (value, temperature, rainfall, humidity)
- y: value scaler로 정규화

In [18]:
# ============================================================
# [정규화] Train 기준 MinMaxScaler (0~1 스케일링)
# ============================================================
# - Train 데이터로만 scaler fit → Val/Test에 동일 scaler 적용 (data leak 방지)
# - scale_cols(4개)만 정규화: value, temperature, rainfall, humidity
# - sin/cos features(6개)는 이미 [0,1] 범위이므로 정규화 불필요
# - normalize_y / denormalize_y: 예측값 ↔ 원본 스케일 변환 함수
# ============================================================

from sklearn.preprocessing import MinMaxScaler

n_features = len(feature_cols)

# --- Train 데이터 기준으로 scaler fit ---
scalers = {}
for col in scale_cols:
    i = feature_cols.index(col)          # feature_cols 내 인덱스
    scaler = MinMaxScaler()
    scaler.fit(X_train[:, :, i].reshape(-1, 1))  # Train의 해당 feature 전체로 fit
    scalers[col] = scaler

# --- X 정규화 함수 (scale_cols만, sin/cos 제외) ---
def normalize_X(arr):
    """X 배열의 scale_cols만 [0,1]로 정규화한다."""
    arr = arr.copy()
    for col in scale_cols:
        i = feature_cols.index(col)
        s = scalers[col]
        d_min, d_max = np.float32(s.data_min_[0]), np.float32(s.data_max_[0])
        arr[:, :, i] = (arr[:, :, i] - d_min) / (d_max - d_min)
    return arr

X_train_scaled = normalize_X(X_train)
X_val_scaled = normalize_X(X_val)
X_test_scaled = normalize_X(X_test)

# --- y 정규화/역정규화 함수 (value scaler 사용) ---
val_min = np.float32(scalers['value'].data_min_[0])
val_max = np.float32(scalers['value'].data_max_[0])

def normalize_y(arr):
    """y값을 [0,1]로 정규화한다."""
    return (arr - val_min) / (val_max - val_min)

def denormalize_y(arr):
    """정규화된 y값을 원본 스케일로 복원한다."""
    return arr * (val_max - val_min) + val_min

y_train_scaled = normalize_y(y_train)
y_val_scaled = normalize_y(y_val)
y_test_scaled = normalize_y(y_test)

print(f"Train 기준 Scaler 범위 (scale_cols만):")
for col in scale_cols:
    s = scalers[col]
    print(f"  {col:>12s}: [{s.data_min_[0]:.2f}, {s.data_max_[0]:.2f}]")
print(f"\nsin/cos features ({n_features - len(scale_cols)}개): 정규화 없이 원본 유지 [0, 1]")

Train 기준 Scaler 범위 (scale_cols만):
         value: [0.00, 310.26]
   temperature: [-11.70, 37.70]
      rainfall: [0.00, 54.00]
      humidity: [4.80, 99.90]

sin/cos features (6개): 정규화 없이 원본 유지 [0, 1]


- Tensor 변환 & DataLoader (pin_memory)

In [19]:
# ============================================================
# [Tensor 변환 & DataLoader] NumPy → PyTorch Tensor → DataLoader
# ============================================================
# - FloatTensor: float32 형태로 GPU 연산 최적화
# - pin_memory=True (CUDA): CPU→GPU 전송 속도 향상 (페이지 잠금 메모리)
# - Train: shuffle=True, drop_last=True (마지막 불완전 배치 제거)
# - Val/Test: shuffle=False (순서 유지), 큰 batch_size(512)로 빠른 평가
# ============================================================

import torch
from torch.utils.data import DataLoader, TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# --- NumPy → PyTorch Tensor ---
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train_scaled)
X_val_tensor = torch.FloatTensor(X_val_scaled)
y_val_tensor = torch.FloatTensor(y_val_scaled)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.FloatTensor(y_test_scaled)

# --- TensorDataset 생성 ---
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# --- DataLoader 생성 ---
batch_size = 256
use_pin = (device.type == 'cuda')  # GPU 사용 시 pin_memory 활성화

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                          drop_last=True, pin_memory=use_pin, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False,
                        pin_memory=use_pin, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False,
                         pin_memory=use_pin, num_workers=0)

print(f"Train: {len(train_loader)} batches | Val: {len(val_loader)} | Test: {len(test_loader)}")

Device: cpu
Train: 564 batches | Val: 61 | Test: 60


## 6. LSTMSeq2SeqAttnModel 생성

In [25]:
# ============================================================
# [모델 정의] LSTMSeq2SeqAttnModel — Encoder-Decoder + Attention
# ============================================================
# 구조:
#   Encoder: 2-layer LSTM → 입력 시퀀스(72 steps)를 hidden state로 압축
#   Attention: Bahdanau(Additive) 방식
#     - Wh(decoder hidden) + Ws(encoder outputs) → tanh → V → softmax
#     - encoder의 72개 time step 중 중요한 부분에 가중치 집중
#   Decoder: LSTMCell × 15 steps (autoregressive)
#     - 각 step마다: attention context + step embedding → LSTMCell → 예측
#     - step_embedding: 디코더가 "현재 몇 번째 step인지" 인식 (위치 정보)
#   출력 헤드: LayerNorm → Dropout → Linear(1)
#
# 파라미터:
#   input_size=10 (features), hidden_size=128, num_layers=2
#   output_size=15 (예측 steps), embed_dim=16, dropout=0.2
# ============================================================

import torch.nn as nn

class LSTMSeq2SeqAttnModel(nn.Module):
    def __init__(self, input_size=10, hidden_size=128, num_layers=2,
                 output_size=15, embed_dim=16, dropout=0.2):
        super().__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers

        # --- Encoder: 2-layer Bidirectional-free LSTM ---
        self.encoder = nn.LSTM(
            input_size=input_size, hidden_size=hidden_size,
            num_layers=num_layers, batch_first=True,
            dropout=dropout if num_layers > 1 else 0  # layer간 dropout
        )

        # --- Step Embedding: 디코더 step 위치 정보 (0~14 → 16dim 벡터) ---
        self.step_embedding = nn.Embedding(output_size, embed_dim)

        # --- Bahdanau Attention ---
        self.attn_Wh = nn.Linear(hidden_size, hidden_size, bias=False)  # decoder query 변환
        self.attn_Ws = nn.Linear(hidden_size, hidden_size, bias=False)  # encoder key 변환
        self.attn_V = nn.Linear(hidden_size, 1, bias=False)             # energy → scalar

        # --- Decoder: LSTMCell (step별 순차 실행) ---
        self.decoder = nn.LSTMCell(
            input_size=hidden_size + embed_dim,  # context(128) + step_emb(16)
            hidden_size=hidden_size
        )

        # --- 출력 헤드: LayerNorm → Dropout → FC ---
        self.layer_norm = nn.LayerNorm(hidden_size)
        self.fc_out = nn.Linear(hidden_size, 1)  # hidden → 유량 1값

    def forward(self, x):
        batch_size = x.size(0)

        # --- Encoder ---
        enc_outputs, (h_n, c_n) = self.encoder(x)      # enc_outputs: (B, 72, 128)
        enc_keys_projected = self.attn_Ws(enc_outputs)  # key 사전 계산 (효율화)

        # Decoder 초기 상태 = Encoder 마지막 layer의 hidden/cell
        h_dec = h_n[-1]  # (B, 128)
        c_dec = c_n[-1]  # (B, 128)

        predictions = []
        self.attn_weights_all = []
        step_ids = torch.arange(self.output_size, device=x.device)
        step_embs = self.step_embedding(step_ids)  # (15, 16)

        # --- Decoder: 15 steps 순차 예측 ---
        for t in range(self.output_size):
            # Attention: query(decoder) × key(encoder) → weight → context
            query = self.attn_Wh(h_dec).unsqueeze(1)             # (B, 1, 128)
            energy = torch.tanh(query + enc_keys_projected)       # (B, 72, 128)
            score = self.attn_V(energy).squeeze(-1)               # (B, 72)
            attn_weights = torch.softmax(score, dim=1)            # (B, 72) 정규화
            context = torch.bmm(attn_weights.unsqueeze(1), enc_outputs).squeeze(1)  # (B, 128)
            self.attn_weights_all.append(attn_weights.detach())

            # Decoder input = attention context + step embedding
            step_emb = step_embs[t].unsqueeze(0).expand(batch_size, -1)  # (B, 16)
            dec_input = torch.cat([context, step_emb], dim=1)            # (B, 144)
            h_dec, c_dec = self.decoder(dec_input, (h_dec, c_dec))

            # 출력: LayerNorm → Dropout → FC
            pred_t = self.fc_out(self.layer_norm(h_dec))   # (B, 1)
            predictions.append(pred_t)

        predictions = torch.cat(predictions, dim=1)                       # (B, 15)
        self.attn_weights_all = torch.stack(self.attn_weights_all, dim=1) # (B, 15, 72)
        return predictions

In [26]:
# [확인] 모델 인스턴스 생성 → 총 파라미터 수 확인 후 삭제
model_name = "Ablation_B"
model_check = LSTMSeq2SeqAttnModel(
    input_size=n_features, hidden_size=128, num_layers=2,
    output_size=output_time, embed_dim=16, dropout=0.2,
)
total_p = sum(p.numel() for p in model_check.parameters())
print(f"Parameters: {total_p:,}")
del model_check  # 메모리 해제 (학습 시 새로 생성)

Parameters: 377,585


## 7. Early Stopping

In [27]:
# ============================================================
# [EarlyStopping] 과적합 방지를 위한 조기 종료 클래스
# ============================================================
# - patience 횟수 동안 val_loss가 min_delta 이상 개선되지 않으면 학습 중단
# - best_model: 가장 낮은 val_loss 시점의 모델 가중치를 저장
# - __call__(): 매 epoch 후 호출 → 개선 시 저장, 미개선 시 카운터 증가
# ============================================================

class EarlyStopping:
    def __init__(self, patience=5, min_delta=1e-5, verbose=True):
        self.patience = patience      # 허용할 연속 미개선 epoch 수
        self.min_delta = min_delta    # 개선으로 인정할 최소 감소량
        self.verbose = verbose
        self.counter = 0              # 연속 미개선 카운터
        self.best_loss: float | None = None
        self.early_stop = False       # True가 되면 학습 루프 탈출
        self.best_model: dict[str, torch.Tensor] | None = None

    def __call__(self, val_loss, model):
        if self.best_loss is None:
            # 첫 호출: 현재 loss를 best로 설정
            self.best_loss = val_loss
            self.save_checkpoint(model)

        elif val_loss < self.best_loss - self.min_delta:
            # 개선됨: best 갱신, 카운터 리셋
            self.best_loss = val_loss
            self.save_checkpoint(model)
            self.counter = 0
            
        else:
            # 미개선: 카운터 증가, patience 초과 시 early_stop 플래그 설정
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter}/{self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True

    def save_checkpoint(self, model):
        """현재 모델 가중치를 best_model에 복사 (deepcopy)."""
        if self.verbose:
            print(f'Validation loss decreased ({self.best_loss:.6f}). Saving model...')
        self.best_model = model.state_dict().copy()

## 8. 학습 (Multi-Run, patience=10)

In [28]:
# ============================================================
# [학습 설정] 하이퍼파라미터 & Step-Weighted MSE Loss
# ============================================================
# - num_epochs=100: 최대 epoch (EarlyStopping으로 조기 종료)
# - learning_rate=0.001: Adam 초기 학습률
# - es_patience=10: EarlyStopping patience
# - step_weights: step 1→1.0, step 15→2.0 (선형 증가)
#   먼 미래일수록 예측이 어려우므로 가중치를 높여 학습 강조
# - weighted_mse_loss: step별 가중 MSE → 단일 scalar loss
# ============================================================

import torch.optim as optim
import time

num_epochs = 100
learning_rate = 0.001
es_patience = 10

# Step-weighted: 가까운 step(1.0) → 먼 step(2.0) 선형 증가 가중치
step_weights = torch.linspace(1.0, 2.0, output_time).to(device)

def weighted_mse_loss(pred, target):
    """Step별 가중치를 곱한 MSE Loss."""
    return (step_weights * (pred - target) ** 2).mean()

criterion = weighted_mse_loss

In [ ]:
# ============================================================
# [Multi-Run 학습] N_RUNS(3)회 반복 학습 + Test 평가
# ============================================================
# 각 Run에서:
#   1) set_seed()로 재현성 보장
#   2) 모델/옵티마이저/스케줄러/EarlyStopping 초기화
#   3) 학습 루프: Train → Val → LR schedule → EarlyStopping 체크
#   4) Best 모델 복원 후 Test 추론
#   5) 원본 스케일로 역정규화 → MAPE/sMAPE/RMSE/MAE/R²/Bias 산출
#   6) Step별 MAPE, 계절별 MAPE, Macro MAPE 기록
#
# 학습 구성:
#   - Optimizer: Adam (lr=0.001, weight_decay=1e-5)
#   - LR Scheduler: ReduceLROnPlateau (factor=0.5, patience=3)
#   - Gradient Clipping: max_norm=1.0
#   - EarlyStopping: patience=10
# ============================================================

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def calc_mape(y_true, y_pred):
    """MAPE(%) — y_true=0인 지점 제외."""
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def calc_smape(y_true, y_pred):
    """sMAPE(%) — 대칭 MAPE, 분모=(|실측|+|예측|)/2."""
    denom = (np.abs(y_true) + np.abs(y_pred)) / 2
    mask = denom > 0
    return np.mean(np.abs(y_true[mask] - y_pred[mask]) / denom[mask]) * 100

all_run_results = []   # 각 run의 평가 결과 dict
all_run_models = []    # 각 run의 best model state_dict
all_run_losses = []    # 각 run의 train/val loss 이력

for run_idx, seed in enumerate(SEEDS[:N_RUNS]):
    set_seed(seed)
    print(f"\n{'='*60}")
    print(f"Run {run_idx+1}/{N_RUNS} (seed={seed})")
    print(f"{'='*60}")
    
    # --- 모델 & 옵티마이저 & 스케줄러 초기화 ---
    model = LSTMSeq2SeqAttnModel(
        input_size=n_features, hidden_size=128, num_layers=2,
        output_size=output_time, embed_dim=16, dropout=0.2,
    ).to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=3  # val_loss 3 epoch 미개선 → lr ×0.5
    )
    early_stopping = EarlyStopping(patience=es_patience, verbose=False)
    
    train_losses, val_losses = [], []
    
    # --- 학습 루프 ---
    for epoch in range(num_epochs):
        # Train phase
        model.train()
        train_loss_epoch = 0.0
        for batch_X, targets in train_loader:
            batch_X, targets = batch_X.to(device), targets.to(device)
            outputs = model(batch_X)
            loss = criterion(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # gradient clipping
            optimizer.step()
            train_loss_epoch += loss.item()
        
        avg_train_loss = train_loss_epoch / len(train_loader)
        train_losses.append(avg_train_loss)
        
        # Validation phase
        model.eval()
        val_loss_epoch = 0.0
        with torch.no_grad():
            for batch_X, targets in val_loader:
                batch_X, targets = batch_X.to(device), targets.to(device)
                outputs = model(batch_X)
                loss = criterion(outputs, targets)
                val_loss_epoch += loss.item()
        
        avg_val_loss = val_loss_epoch / len(val_loader)
        val_losses.append(avg_val_loss)
        
        # LR Scheduler & 로깅
        scheduler.step(avg_val_loss)
        current_lr = optimizer.param_groups[0]['lr']
        
        if (epoch + 1) % 10 == 0:
            print(f'  Epoch [{epoch+1:3d}/{num_epochs}] '
                  f'Train: {avg_train_loss:.6f} '
                  f'Val: {avg_val_loss:.6f} '
                  f'LR: {current_lr:.6f}')
        
        # EarlyStopping 체크
        early_stopping(avg_val_loss, model)
        if early_stopping.early_stop:
            print(f"  Early Stopping at Epoch {epoch+1} "
                  f"(Best Val Loss: {early_stopping.best_loss:.6f})")
            break
    
    # --- Best 모델 복원 ---
    if early_stopping.best_model is not None:
        model.load_state_dict(early_stopping.best_model)
    
    all_run_models.append(copy.deepcopy(model.state_dict()))
    all_run_losses.append({'train': train_losses, 'val': val_losses})
    
    # --- Test 추론 & 역정규화 ---
    model.eval()
    test_preds_list, test_actuals_list = [], []
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X = batch_X.to(device)
            outputs = model(batch_X)
            test_preds_list.append(outputs.cpu().numpy())
            test_actuals_list.append(batch_y.numpy())
    
    run_preds = denormalize_y(np.vstack(test_preds_list))     # 원본 스케일 복원
    run_actuals = denormalize_y(np.vstack(test_actuals_list))
    
    # --- 전체 지표 산출 ---
    run_result = {
        'seed': seed,
        'epoch': len(train_losses),
        'mape': calc_mape(run_actuals.flatten(), run_preds.flatten()),
        'smape': calc_smape(run_actuals.flatten(), run_preds.flatten()),
        'rmse': np.sqrt(mean_squared_error(run_actuals, run_preds)),
        'mae': mean_absolute_error(run_actuals, run_preds),
        'r2': r2_score(run_actuals.flatten(), run_preds.flatten()),
        'bias': float(np.mean(run_actuals - run_preds)),  # 양수=과소예측
        'predictions': run_preds,
        'actuals': run_actuals,
    }
    
    # --- Step별 MAPE (1~15분 각각) ---
    step_mapes = []
    for s in range(output_time):
        step_mapes.append(calc_mape(run_actuals[:, s], run_preds[:, s]))
    run_result['step_mapes'] = step_mapes
    
    # --- 계절별 MAPE (block 단위) ---
    season_mapes = []
    offset = 0
    for size in [len(arr) for arr in X_test_list]:
        block_a = run_actuals[offset:offset+size]
        block_p = run_preds[offset:offset+size]
        season_mapes.append(calc_mape(block_a.flatten(), block_p.flatten()))
        offset += size
    run_result['season_mapes'] = season_mapes
    run_result['macro_mape'] = np.mean(season_mapes)  # 계절 균등 평균
    
    all_run_results.append(run_result)
    
    print(f"  → MAPE: {run_result['mape']:.2f}% | SMAPE: {run_result['smape']:.2f}% "
          f"| R²: {run_result['r2']:.4f} | Bias: {run_result['bias']:+.2f} "
          f"| Epoch: {run_result['epoch']}")

print(f"\n{'='*60}")
print("All runs completed")
print(f"{'='*60}")


Run 1/3 (seed=42)
  Epoch [ 10/100] Train: 0.001550 Val: 0.001298 LR: 0.001000
  Epoch [ 20/100] Train: 0.001288 Val: 0.001120 LR: 0.000500


## 9. 평가

### Step별 정확도 (특정 시점 조회)
- `step_accuracy(step)`: 특정 예측 시점(1~15분)의 정확도 지표 반환
- 정확도 = `100 - MAPE(%)`

In [ ]:
# ============================================================
# [평가] Best Run 선택 & Step별 정확도 함수
# ============================================================
# - MAPE가 가장 낮은 run을 best로 선택
# - step_accuracy(step): 특정 예측 시점(1~15분)의 정확도 지표 계산
#   정확도 = 100 - MAPE(%)
# - 전체 Step 요약 테이블 출력
# ============================================================

best_idx = int(np.argmin([r['mape'] for r in all_run_results]))
best_run = all_run_results[best_idx]
preds = best_run['predictions']   # (n_test, 15) 역정규화된 예측값
actuals = best_run['actuals']     # (n_test, 15) 역정규화된 실측값

print(f"Best Run: seed={best_run['seed']} (Run {best_idx+1})")
print(f"Test samples: {len(preds):,}\n")

def step_accuracy(step):
    """
    특정 예측 시점(step)의 정확도 지표를 계산한다.

    Parameters
    ----------
    step : int
        조회할 예측 시점 (1~15). step=1 → +1분, step=15 → +15분

    Returns
    -------
    dict  {'step', 'time', 'accuracy', 'mape', 'smape', 'rmse', 'mae', 'r2'}
        - accuracy = 100 - MAPE (%)
    """
    assert 1 <= step <= actuals.shape[1], f"step 범위: 1~{actuals.shape[1]}"
    a = actuals[:, step - 1]
    p = preds[:, step - 1]

    mape  = calc_mape(a, p)
    smape = calc_smape(a, p)
    rmse  = np.sqrt(mean_squared_error(a, p))
    mae   = mean_absolute_error(a, p)
    r2    = r2_score(a, p)

    return {
        'step': step,
        'time': f'+{step}min',
        'accuracy': 100 - mape,
        'mape': mape,
        'smape': smape,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
    }

# --- 전체 Step 요약 테이블 ---
rows = [step_accuracy(s) for s in range(1, output_time + 1)]
df_steps = pd.DataFrame(rows).set_index('step')

print("=" * 72)
print(f"  Step별 정확도  (Best Run, seed={best_run['seed']})")
print("=" * 72)
print(df_steps.to_string(float_format='%.2f'))
print(f"\n전체 평균 정확도: {df_steps['accuracy'].mean():.2f}%")
print(f"전체 MAPE:       {best_run['mape']:.2f}%")

In [ ]:
# ============================================================
# [특정 시점 조회] query_step 변경으로 원하는 시점의 정확도 확인
# ============================================================
# 사용법: query_step = 원하는 값(1~15)으로 변경 후 셀 실행
# 예) query_step = 1  → +1분 시점
#     query_step = 5  → +5분 시점
#     query_step = 15 → +15분 시점
# ============================================================

query_step = 15

result = step_accuracy(query_step)

print(f"━━━ +{query_step}min 시점 예측 정확도 ━━━")
print(f"  정확도 (100-MAPE) : {result['accuracy']:.2f}%")
print(f"  MAPE              : {result['mape']:.2f}%")
print(f"  sMAPE             : {result['smape']:.2f}%")
print(f"  RMSE              : {result['rmse']:.2f}")
print(f"  MAE               : {result['mae']:.2f}")
print(f"  R²                : {result['r2']:.4f}")

In [ ]:
# ============================================================
# [모델 저장] Best Run의 모델 가중치 + 메타데이터를 .pth로 저장
# ============================================================
# 저장 항목:
#   - model_state_dict: 학습된 모델 가중치
#   - scalers: 정규화 범위 (추론 시 동일 스케일링 적용 필요)
#   - feature_cols / scale_cols: 입력 컬럼 구성
#   - input_time / output_time: 입출력 시퀀스 길이
#   - train/val losses, best seed/epoch: 학습 이력
# ============================================================

model_path = BASE_DIR / "models" / "ablation_b_dropout_removed.pth"

torch.save({
    'model_name': model_name,
    'model_state_dict': model.state_dict(),
    'scalers': {col: {'min': scalers[col].data_min_[0], 'max': scalers[col].data_max_[0]} for col in scale_cols},
    'feature_cols': feature_cols,
    'scale_cols': scale_cols,
    'train_losses': train_losses,
    'val_losses': val_losses,
    'test_loss': best_run['mape'],
    'best_epoch': len(train_losses),
    'best_seed': best_run['seed'],
    'input_time': input_time,
    'output_time': output_time,
}, model_path)

print(f"모델 저장 완료: {model_path}")